In [5]:
import re
import torch
from sklearn.model_selection import train_test_split         # разбить данные на тестовые и тренеровочные
import warnings
warnings.filterwarnings("ignore")


# Строим класс RNN который будет принимать различную указанную вариацию рекуррентной ячейки - GRU / LSTM/ SimpleRNN
class RnnFlex(torch.nn.Module):
                        # тип     размер словаря  размер эмб       скрытые слои   классы
    def __init__(self, rnnClass, dictionary_size, embedding_size, num_hiddens, num_classes):
        super().__init__()
        self.num_hiddens = num_hiddens
        self.embedding = torch.nn.Embedding(dictionary_size, embedding_size) # учится представлять наши выходные параметры в виде векторов
        self.hidden = rnnClass(embedding_size, num_hiddens, batch_first=True) # batch_first=True когды мы начинаем с нулевого хидден стейта
        self.output = torch.nn.Linear(num_hiddens, num_classes)

    def forward(self, X):
        out = self.embedding(X)  # прошкалировали закодированный Х и добавили размерность 1,40, 64
        #print(out.shape)
        _, state = self.hidden(out)  # приходят все выходы и последний выход (между LSTM  и GRU выход немного разный)
        #print(state[0])
        predictions = self.output(state[0])
        return predictions

def sample(preds):
    softmaxed = torch.softmax(preds, 1) # распределяем значения от 0 до 1
    #print('softmaxed  ',softmaxed)
    probas = torch.distributions.multinomial.Multinomial(1, softmaxed).sample()  # нормализация
    #print('probas   ',probas)
    #print(probas.max(dim=1)[1])
    return probas.max(dim=1)[1]#.argmax()


# Функция шифрования с тремя параметрами: текст, ключ, язык
def ceaser_cipher(user, key, lang):
    # Переменная результата шифрования; переменная, определяющая верхний и нижний регистр
    res, n = [], ""

    # Проверка пользователем выбранного языка

    # Проверка выбран ли русский язык (регистр букв, вводимых пользователем, не важен)
    if lang.lower() in ["русский", "russian"]:
        # Двум переменным присваиваются русская азбука нижнего и верхнего регистра соответственно
        dictionary, dictionary_upper = "абвгдеёжзийклмнопрстуфхцчшщъыьэюя", "АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ"
    # Проверка выбран ли английский язык язык (регистр букв, вводимых пользователем, не важен)
    elif lang.lower() in ["английский", "english"]:
        # Двум переменным присваиваются английской азбука нижнего и верхнего регистра соответственно
        dictionary, dictionary_upper = "abcdefghijklmnopqrstuvwxyz", "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    else:
        return "Такого языка нет в опции"

    # Цикл проверки, где каждую итерацию будет обрабатываться один символ из текста последовательно
    for i in range(len(user)):
        # Проверка символа на верхний или нижний регистр

        # Принадлежит ли символ нижнему регистру
        if user[i] in dictionary:
            n = dictionary
        # Принадлежит ли символ верхнему регистру
        elif user[i] in dictionary_upper:
            n = dictionary_upper
        # Символ не принадлежит ни нижнему ни верхнему регистру (символ не является буквой)
        else:
            res.append(user[i])

        # Если символ есть в списке n (является буквой), то будет происходить его зашифровка
        if user[i] in n:
            # Цикл перебора азбуки
            for j in range(len(n)):
                # Если порядковый номер буквы + ключ находятся  в диапазоне от 0 до конца азбуки
                # и если буква из текста совпадает с буквой из азбуки, то:
                if 0 <= j + key < len(n) and user[i] == n[j]:
                    # В результат добавляется буква со сдвигом key (зашифрованная буква)
                    res.append(n[j + key])
                # Если порядковый номер буквы + ключ выходит из диапазона азбуки, превышая его
                # и если буква из текста совпадает с буквой из азбуки, то:
                elif j + key >= len(n) and user[i] == n[j]:
                    # В результат добавляеться буква со сдвигом key,
                    # при этом преводя порядковый номер буквы к диапазону азбуки (зашифрованая буква)
                    res.append(n[(1 - j - key) % (len(n) - 1)])
                # Если порядковый номер буквы + ключ выходит из диапазона азбуки, недотягивает до него
                # и если буква из текста совпадает с буквой из азбуки, то:
                elif j + key < 0 and user[i] == n[j]:
                    # В результат добавляеться буква со сдвигом key,
                    # при этом преводя порядковый номер буквы к диапазону азбуки (зашифрованая буква)
                    res.append(n[(j + key) % len(n)])

    # Функция возвращает зашифрованный текст
    return ''.join(res)

text = 'а б в г д е ё ж з и й к л м н о п р с т у ф х ц ч ш щ ъ ы ь э ю я \
    Шифр Цезаря, также известный как шифр сдвига, код Цезаря — один из самых простых и наиболее широко известных методов шифрования.\
Шифр Цезаря — это вид шифра подстановки, в котором каждый символ в открытом тексте заменяется символом, находящимся на некотором \
постоянном числе позиций левее или правее него в алфавите. Например, в шифре со сдвигом вправо на 3, А была бы заменена на Г, Б \
станет Д, и так далее.Шифр назван в честь римского полководца Гая Юлия Цезаря, использовавшего его для секретной переписки со своими \
генералами.Шаг шифрования, выполняемый шифром Цезаря, часто включается как часть более сложных схем, таких как шифр Виженера, и \
всё ещё имеет современное приложение в системе ROT13. Как и все моноалфавитные шифры, шифр Цезаря легко взламывается и не имеет \
почти никакого применения на практике.Шифр Цезаря называют в честь Юлия Цезаря, который, согласно «Жизни двенадцати цезарей» \
Светония, использовал его со сдвигом 3,чтобы защищать военные сообщения. Хотя Цезарь был первым зафиксированным человеком, использовавшим эту схему, другие шифры \
 подстановки, как известно, использовались и ранее.Если у него было что-либо конфиденциальное для передачи, то он записывал это \
  шифром, то есть так изменял порядок букв алфавита, что нельзя было разобрать ни одно слово. Если кто-либо хотел дешифровать его и \
 понять его значение, то он должен был подставлять четвертую букву алфавита, а именно, D, для A, и так далее, с другими буквами.\
 Гай Светоний Транквилл Жизнь двенадцати цезарей, Книга первая, гл. 56[3] Его племянник, Август, также использовал этот шифр, \
 но со сдвигом вправо на один, и он не повторялся к началу алфавита:Всякий раз, когда он записывал шифром, он записал B для A,\
 C для B, и остальной части букв на том же самом принципе, используя AA для X.Гай Светоний Транквилл Жизнь двенадцати цезарей,\
 Книга вторая, гл. 88[3] Есть доказательства, что Юлий Цезарь использовал также и более сложные схемы[4].Неизвестно, насколько \
 эффективным шифр Цезаря был в то время, но, вероятно, он был разумно безопасен, не в последнюю очередь благодаря тому, что \
 большинство врагов Цезаря было неграмотным, и многие предполагали, что сообщения были написаны на неизвестном иностранном языке[5]. \
 Нет никаких свидетельств того времени касательно методов взлома простых шифров подстановки. Самые ранние сохранившиеся записи о \
 частотном анализе — это работы Ал-Кинди 9-го века об открытии частотного анализа[6].Шифр Цезаря со сдвигом на один используется на \
 обратной стороне мезузы, чтобы зашифровать имена Бога. Это может быть пережитком с раннего времени, когда еврейскому народу не \
 разрешили иметь мезузы[7].В XIX веке личная секция рекламных объявлений в газетах иногда использовалась, чтобы обмениваться \
 сообщениями, зашифрованными с использованием простых шифров. Кан (1967) описывает случаи, когда любители участвовали в секретных \
 коммуникациях, зашифрованных с использованием шифра Цезаря в «Таймс»[8]. Даже позднее, в 1915, шифр Цезаря находил применение: \
 российская армия использовала его как замену для более сложных шифров, которые оказались слишком сложными для войск; у немецких\
 и австрийских криптоаналитиков были лишь небольшие трудности в расшифровке этих сообщений[9].Шифр Цезаря со сдвигом тринадцать\
 также используется в алгоритме ROT13, простом методе запутывания текста, широко используемом в Usenetе, и используется скорее \
 как способ сокрытия спойлеров, чем как метод шифрования[10]. Шифр Виженера использует шифр Цезаря с различными сдвигами в каждой \
 позиции в тексте; значение сдвига определяется с помощью повторяющегося ключевого слова. Если ключевое слово такое же длинное, \
 как и сообщение, сгенерировано случайным образом, содержится в тайне и используется лишь однократно — такая схема называется схема \
 одноразовых блокнотов — и это единственная система шифрования, для которой доказана абсолютная криптографическая стойкость[11].\
 Ключевые слова короче, чем сообщение (например, «Complete Victory», использовавшееся Конфедерацией во время гражданской войны в США)\
 , вводят циклический образец, который мог бы быть обнаружен с помощью улучшенной версии частотного анализа[12].В апреле 2006 беглый \
 босс Мафии Бернардо Провенцано был пойман в Сицилии частично из-за криптоанализа его сообщений, написанных с\
 использованием вариации шифра Цезаря. В шифре Провенцано буквы сначала заменялись на числа — порядковые номера букв в алфавите, \
 а уже к полученной последовательности чисел применялся шифр Цезаря — так, чтобы при сдвиге на 3 «A» была написана как «4», «B» —\
 как «5», и так далее[13].Часто для удобства использования шифра Цезаря используют два насаженных на общую ось диска разного \
 диаметра с нарисованными по краям дисков алфавитами. Изначально диски поворачиваются так, чтобы напротив каждой буквы алфавита \
 внешнего диска находилась та же буква алфавита малого диска. Если теперь повернуть внутренний диск на несколько символов, то \
 мы получим соответствие между символами внешнего диска и внутреннего — шифр Цезаря. Получившийся диск можно использовать как \
 для шифрования, так и для расшифровки[14].Например, если внутреннее колесо повернуть так, чтобы символу A внешнего диска \
 соответствовал символ D внутреннего диска, то мы получим шифр со сдвигом 3 влево. ю я ю я ю я ю'.lower() 


# Число для сдвига
k = 3 

# язык текста, который будет зашифрован
language = 'русский' 

# Вывод зашифрованного текста
text = re.sub('[a-zA-Z\s]+|[^\w\s]+|[\d]+', ' ', text)
shifr = ceaser_cipher(text, k, language)
#print(shifr)


# Создаем индексы под символы
INDEX_TO_CHAR = sorted(list(set(text.lower())))

# Кодировщик для символа в индекс:
CHAR_TO_INDEX = {c: i for i, c in enumerate(INDEX_TO_CHAR)}


# Необходимо теперь нам это все превратить в тензоры для того чтобы подать в алгоритм рекуррентной нейронной сети
X = torch.zeros((len(shifr), 1), dtype=int)
Y = torch.zeros((len(text)), dtype=int)

# Пробегаемся по нашим кусочкам предложений и кодируем под сформированные символы
i = 0
for char_s, char_t in zip(shifr, text.lower()):
    #print(i,char_s, char_t )
    X[i,0] = CHAR_TO_INDEX[char_s]
    Y[i] = CHAR_TO_INDEX[char_t]
    i  = i + 1

#разбиваем матрицу на тестовую, тренировачную и валидационную 
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    Y, 
    test_size=0.30, 
    random_state=42)

X_valid, X_test, y_valid, y_test = train_test_split(
    X_test, 
    y_test, 
    test_size=0.5, 
    random_state=42)

#print(X_train.shape, y_train.shape)
#print(X_test.shape, y_test.shape)

BATCH_SIZE = 16  #16 наблюдений (строки)

dataset = torch.utils.data.TensorDataset(X_train, y_train)
data_train = torch.utils.data.DataLoader(dataset, BATCH_SIZE, shuffle=True)

dataset = torch.utils.data.TensorDataset(X_test, y_test)
data_test = torch.utils.data.DataLoader(dataset, BATCH_SIZE, shuffle=True)

dataset = torch.utils.data.TensorDataset(X_valid, y_valid)
data_valid = torch.utils.data.DataLoader(dataset, BATCH_SIZE, shuffle=True)

model = RnnFlex(torch.nn.RNN, len(CHAR_TO_INDEX), 64, 128, len(CHAR_TO_INDEX))

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

for ep in range(100):

    train_loss = 0.
    train_passed = 0
    train_acc_sum = 0
    n_train = 0

    model.train()
    for X_b, y_b in data_train:
        #print(type(X_b), type(y_b))
        optimizer.zero_grad()
        answers = model(X_b)
        #answers = answers.view(-1, len(INDEX_TO_CHAR))
        #print(X_b, y_b, answers)
        #print(y_b.shape)
        #print(answers.shape, y_b.shape)
        predicted = sample(answers)
        #print(X_b.shape, y_b.shape, answers.shape, predicted.shape)
        train_acc_sum += (predicted == y_b).sum().item()
        n_train += y_b.shape[0]

        loss = criterion(answers, y_b)

        #print(predicted)
        #print(y_b)
        #print(train_acc_sum)
        #print(y_b.shape[0])

        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_passed += 1

    model.eval()

    test_acc_sum = 0
    n_test = 0

    for X_test, y_test in data_test:
        #print(X_test.shape, y_test.shape)
        answers = model(X_test) 
        #print(predicted.shape)
        predicted = sample(answers)
        test_acc_sum += (predicted == y_test).sum().item()
        n_test += y_test.shape[0]

    if ep % 10 == 0:
        print("Epoch {}    Train Loss: {:.3f}  Train acc: {:.3f}  Test acc: {:.3f}".format(ep, train_loss / train_passed, train_acc_sum / n_train, test_acc_sum / n_test))


valid_acc_sum = 0
n_valid = 0

for X_valid, y_valid in data_valid:
    #print(X_test.shape, y_test.shape)
    answers = model(X_valid) 
    #print(predicted.shape)
    predicted = sample(answers)
    valid_acc_sum += (predicted == y_valid).sum().item()
    n_valid += y_valid.shape[0]

    shifr='' 
    for idx in X_valid:
        shifr += INDEX_TO_CHAR[idx]

    deshifr = ''
    for idx in predicted:
        deshifr += INDEX_TO_CHAR[idx]  
        
    print('shifr: {}       deshifr: {}'.format(shifr, deshifr))

print("Valid acc: {:.3f}".format(valid_acc_sum / n_valid))   

Epoch 0    Train Loss: 0.614  Train acc: 0.721  Test acc: 0.950
Epoch 10    Train Loss: 0.013  Train acc: 0.994  Test acc: 0.992
Epoch 20    Train Loss: 0.014  Train acc: 0.993  Test acc: 0.992
Epoch 30    Train Loss: 0.013  Train acc: 0.991  Test acc: 0.991
Epoch 40    Train Loss: 0.013  Train acc: 0.993  Test acc: 0.992
Epoch 50    Train Loss: 0.013  Train acc: 0.994  Test acc: 0.987
Epoch 60    Train Loss: 0.013  Train acc: 0.992  Test acc: 0.991
Epoch 70    Train Loss: 0.013  Train acc: 0.993  Test acc: 0.986
Epoch 80    Train Loss: 0.013  Train acc: 0.993  Test acc: 0.992
Epoch 90    Train Loss: 0.013  Train acc: 0.992  Test acc: 0.991
shifr: е омуэсдгзрослзз       deshifr: в лйряобаенлоиее
shifr: п   нфж ър хфд п       deshifr: м   ксд чн тсб м
shifr: ёлх   фэ  зслгмо       deshifr: гит   ся  еоиайл
shifr: фяозрх зе гзсллс       deshifr: сьлент ев аеоиио
shifr: ък гхнорлюъ ц ск       deshifr: чз атклниыч у оз
shifr:    згсотжх знрпю       deshifr:    еаолпдт екнмы
shifr: ггюфсея 

In [ ]:
"""
б в г д е ё ж з и й к л м н о п р с т у ф х ц ч ш щ ъ ы ь э ю я а   # k = 1
в г д е ё ж з и й к л м н о п р с т у ф х ц ч ш щ ъ ы ь э ю я а б   # k = 2
г д е ё ж з и й к л м н о п р с т у ф х ц ч ш щ ъ ы ь э ю я а б в   # k =3
а б в г д е ё ж з и й к л м н о п р с т у ф х ц ч ш щ ъ ы ь э ю я 
"""


In [4]:
# Для К = 1
test_shifr_1 = 'Ф мфлпнпсэа ефв иёмжоьк; Имбуба чёрэ об ефвё упн: Й еожн й опшэя лпу фшжоьк Гтж цпейу рп чёрй лсфдпн; Йежу обрсбгп — рётоэ ибгпейу, Обмёгп — тлбилф дпгпсйу. Убн шфеётб: убн мёщйк вспейу, Сфтбмлб об гёугац тйейу; Убн об оёгёепньц епспзлбц Тмёеь оёгйебооьц игёсёк; Йивфщлб убн об лфсэйц опзлбц Тупйу вёи плпо, вёи егёсёк; Убн мёт й епм гйеёойк рпмоь; Убн п ибсё рсйцмьофу гпмоь Об всёд рётшбоьк й рфтупк, Й усйечбуэ гйуаиёк рсёлсбтоьц Шсёепк йи гпе гьцпеау атоьц, Й т ойнй еаеэлб йц нпстлпк; Убн лпспмёгйш нйнпцпепн Рмёоаёу дспиопдп чбса; Убн г пвмблбц рёсёе обспепн Шёсёи мётб, шёсёи нпса Лпмефо оётжу впдбуьса; Г уёнойчё убн чбсёгоб уфзйу, Б вфськ гпмл ёк гёсоп тмфзйу; Убн туфрб т Вбвпя Адпк Йежу, всёежу тбнб тпвпк, Убн чбсэ Лбъёк обе имбупн шбцоёу;'.lower()
# Для К = 2
test_shifr_2 = 'Х нхмрортюб ёхг йжнзпэл; Йнвфвб шжсю пв ёхгж фро: К ёпзо к прщюа мрф хщзпэл Дуз чрёкф ср шжск мтхеро; Кёзф пвствдр — сжупю йвдрёкф, Пвнждр — умвймх ердрткф. Фво щхёжув: фво нжъкл гтрёкф, Тхувнмв пв джфдбч укёкф; Фво пв пжджёроэч ёртримвч Унжёэ пждкёвппэч йджтжл; Кйгхъмв фво пв мхтюкч примвч Уфркф гжй рмрп, гжй ёджтжл; Фво нжу к ёрн дкёжпкл срнпэ; Фво р йвтж сткчнэпхф дрнпэ Пв гтже сжущвпэл к схуфрл, К фткёшвфю дкфбйжл стжмтвупэч Щтжёрл кй дрё дэчрёбф бупэч, К у пкок ёбёюмв кч ортумрл; Фво мртрнждкщ окорчрёро Снжпбжф етрйпрер швтб; Фво д ргнвмвч сжтжё пвтрёро Щжтжй нжув, щжтжй ортб Мрнёхп пжузф гревфэтб; Д фжопкшж фво швтждпв фхикф, В гхтэл дрнм жл джтпр унхикф; Фво уфхсв у Гвгра Берл Кёзф, гтжёзф увов ургрл, Фво швтю Мвыжл пвё йнвфро щвчпжф;'.lower()
# Для К = 3
test_shifr_3 = 'Ц оцнспсуяв жцд кзоирюм; Когхгв щзтя рг жцдз хсп: Л жрип л рсъяб нсх цъирюм Ефи шсжлх тс щзтл нуцёсп; Лжих ргтугес — тзфря кгесжлх, Ргозес — фнгкнц ёсесулх. Хгп ъцжзфг: хгп озылм дусжлх, Уцфгонг рг езхевш флжлх; Хгп рг рзезжспюш жсусйнгш Фозжю рзелжгррюш кезузм; Лкдцынг хгп рг нцуялш рсйнгш Фхслх дзк снср, дзк жезузм; Хгп озф л жсо елжзрлм тсорю; Хгп с кгуз тулшоюрцх есорю Рг дузё тзфъгрюм л тцфхсм, Л хулжщгхя елхвкзм тузнугфрюш Ъузжсм лк есж еюшсжвх вфрюш, Л ф рлпл жвжянг лш псуфнсм; Хгп нсусозелъ плпсшсжсп Тозрвзх ёускрсёс щгув; Хгп е сдогнгш тзузж ргусжсп Ъзузк озфг, ъзузк псув Нсожцр рзфих дсёгхюув; Е хзпрлщз хгп щгузерг хцйлх, Г дцуюм есон зм езурс фоцйлх; Хгп фхцтг ф Дгдсб Вёсм Лжих, дузжих фгпг фсдсм, Хгп щгуя Нгьзм ргж когхсп ъгшрзх;'.lower()


test_shifr = re.sub('[^\w\s]', ' ', test_shifr_3)
X_pred = torch.zeros((len(test_shifr), 1), dtype=int)

# Пробегаемся по нашим кусочкам предложений и кодируем под сформированные символы
for n, char in enumerate(test_shifr):
    X_pred[n,0] = CHAR_TO_INDEX[char]

preds = model(X_pred)

deshifr = ''

for ch in sample(preds):
    deshifr += INDEX_TO_CHAR[ch]
print(deshifr)

у лукоморьы дуб зелёный  златая цепь на дубе том  и днём и ночьб кот учёный всё ходит по цепи кругом  идёт направо   песнь заводит  налево   сказку говорит  там чудеса  там леший бродит  русалка на ветвях сидит  там на неведомых дорожках следы невиданных зверей  избушка там на курьих ножках стоит без окон  без дверей  там лес и дол видений полны  там о заре прихлынут волны на брег песчаный и пустой  и тридцать витгзей прекрасных чредой из вод выходят ясных  и с ними дядька их морской  там королевич мимоходом пленыет грозного царя  там в облаках перед народом через леса  через моря колдун несёт богатыря  в темнице там царевна тужит  а бурый волк ей верно служит  там ступа с бабох ыгой идёт  бредёт сама собой  там царь кащей над златом чахнет 
